In [1]:
# pip install pyspark
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext.getOrCreate()
# initializing spark session
spark = SparkSession(sc) 

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/07 11:51:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
creditcardRDD = sc.textFile("SalesTraining.csv")    # Reading the file from DBFS location

# display record count - before removing the header record
print("Total records including header : ", creditcardRDD.count())

header = creditcardRDD.first() # extract the header record from the RDD
dataRDD = creditcardRDD.filter(lambda row: row != header)   # use 'filter' (transformation) to remove the header

# display record count - after removing the header record
print("Total records including header : ", dataRDD.count())

# split the record by ',' and select only the first 3 columns - Region, Country and Item Type
tupleRDD = dataRDD.map(lambda each_row: each_row.split(',')).map(lambda each_row: (each_row[0], each_row[1], each_row[2])) 

# convert the RDD to a DataFrame, display first 5 rows with no truncation of column data
columns = ["Region", "Country", "ItemType"]

# convert the RDD to DataFrame. Show() is as action, which shows 1st 5 records with record truncate=False
tupleRDD.toDF(columns).show(5, False)

Total records including header :  950
Total records including header :  949
+------------------+----------+----------+
|Region            |Country   |ItemType  |
+------------------+----------+----------+
|Sub-Saharan Africa|Burundi   |Vegetables|
|Europe            |Ukraine   |Cosmetics |
|Europe            |Croatia   |Beverages |
|Sub-Saharan Africa|Madagascar|Fruits    |
|Asia              |Malaysia  |Snacks    |
+------------------+----------+----------+
only showing top 5 rows



In [3]:
# convert the tuple RDD to DataFrame with header record
creditcardDF = tupleRDD.toDF(columns)

# display the schema
creditcardDF.printSchema()

root
 |-- Region: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- ItemType: string (nullable = true)



In [4]:
# display the columns
creditcardDF.columns

# display the RDD data - top 3 records
creditcardDF.take(3)

[Row(Region='Sub-Saharan Africa', Country='Burundi', ItemType='Vegetables'),
 Row(Region='Europe', Country='Ukraine', ItemType='Cosmetics'),
 Row(Region='Europe', Country='Croatia', ItemType='Beverages')]

In [5]:
# %scala code 
# val rdd=sc.parallelize(0 to 10,1)
# val last=rdd.sortBy(x=>{x},false,1).first()
# println(last)